In [2]:
search_for = 130
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0023431777954101562
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 749112551
type: [1, 1, 1, 1, 130] 130
cases of this type: 285610000
100000 30.24381059291419
200000 51.850348681473676
300000 58.24581338239283
400000 54.91627488140278
500000 57.3788359298263
600000 58.74457467006457
700000 55.97086588522511
800000 56.237960271639324
900000 54.348689828237106
1000000 56.185879512190986
1100000 38.858973909025
1200000 32.74080276253867
1300000 55.187218832810025
1400000 53.936156794583646
1500000 54.747253437794484
1600000 54.66454256571467
1700000 58.13370839805777
1800000 55.56236999703381
1900000 54.0670865585701
2000000 56.164180008699354
2100000 54.971849058936506
2200000 43.211629959849326
2300000 29.047829863394938
2400000 53.62937570808499
2500000 55.33322336242582
2600000 52.34816356405733
2700000 54.657419568306075
2800000 56.03320813413202
2900000 54.007584718902045
3000000 53.22729

29800000 50.880794508436416
29900000 58.876448846284255
30000000 56.76115989724317
30100000 62.014881951565805
30200000 59.46139127135157
30300000 58.193199122868336
30400000 65.48921903179276
30500000 58.88064331080253
30600000 24.823413224346634
30700000 62.566851795031816
30800000 65.32940497213254
30900000 71.97767276188624
31000000 42.27969574362026
31100000 43.00388966783989
31200000 67.9144849977212
31300000 63.693759655156136
31400000 69.63841861199876
31500000 59.67514727197578
31600000 66.67345450461042
31700000 62.06842457829627
31800000 58.94721481634053
31900000 19.3282021433131
32000000 54.97164922404805
32100000 64.41707680507244
32200000 61.288628636352065
32300000 62.83308469751982
32400000 69.09240186895356
32500000 61.08513864247612
32600000 70.38547526454894
32700000 54.01993948914681
32800000 32.316355354085985
32900000 66.08186230578833
33000000 66.43179100914405
33100000 69.52187364671222
33200000 48.82614734737686
33300000 34.59706970687052
33400000 66.680667773

60000000 57.46784222055198
60100000 56.84163727843935
60200000 49.481348590366316
60300000 56.38980330020578
60400000 42.96929531720022
60500000 22.84014597863591
60600000 52.82137057443466
60700000 51.394765513108794
60800000 54.42579275917158
60900000 51.31564225699649
61000000 60.05780260267939
61100000 25.5703113667284
61200000 54.54550413108283
61300000 63.58421360047785
61400000 55.58581460066103
61500000 64.19580844554407
61600000 59.7461650593617
61700000 60.16559926804828
61800000 59.5663076502261
61900000 58.27195170518775
62000000 40.1223658552582
62100000 39.51644834541401
62200000 57.445950882480865
62300000 53.7956501979628
62400000 48.1861234327274
62500000 54.00849128704308
62600000 42.33544358911442
62700000 24.687062598805163
62800000 52.86873509874939
62900000 50.809559768726
63000000 52.26459513097149
63100000 51.93279541106456
63200000 57.79066251535172
63300000 23.80211231495173
63400000 57.54037287726566
63500000 61.36448153161779
63600000 56.98243440029683
63700

90200000 50.7098296735315
90300000 50.96704802861455
90400000 52.14838712655479
90500000 52.33409372011574
90600000 47.61927372758094
90700000 45.626644052122636
90800000 25.690543584121592
90900000 38.732792279850344
91000000 48.09002594375683
91100000 45.878588696427514
91200000 25.82793350176568
91300000 40.514645387569146
91400000 45.86392551121166
91500000 46.134429629394596
91600000 29.545060815014686
91700000 38.037973970040156
91800000 45.88563646635553
91900000 48.56222619820537
92000000 49.966878233642376
92100000 53.26948695981429
92200000 49.23900761188315
92300000 51.7851843172539
92400000 55.23332333317696
92500000 53.45126329050702
92600000 55.89775110529756
92700000 53.48144576870371
92800000 51.073978698128826
92900000 50.47294160660258
93000000 31.61601525313967
93100000 37.82057413306377
93200000 47.7423821244604
93300000 49.500027421365196
93400000 24.770328019483355
93500000 40.88698243460441
93600000 49.58275896075973
93700000 49.17610787251
93800000 25.1134702457

119600000 17.88510420833904
119700000 49.479380969319145
119800000 13.907745434000383
119900000 46.86484162654894
120000000 24.0868038210966
120100000 52.00952017436379
120200000 48.260661176795
120300000 54.16852477589522
120400000 48.493323848071164
120500000 54.036365486961074
120600000 51.066622707637016
120700000 54.03447037842323
120800000 61.46935702430692
120900000 55.1851906864674
121000000 59.74486418881257
121100000 51.2498189110113
121200000 48.888302540417534
121300000 52.54937901394321
121400000 47.61553681870115
121500000 54.417339846857544
121600000 49.00288658071723
121700000 48.09516659364709
121800000 22.542651373252706
121900000 44.50406074306066
122000000 14.115160605590681
122100000 37.857504150799386
122200000 29.743633817161847
122300000 49.14540251437961
122400000 48.7298079043274
122500000 54.49065267328926
122600000 47.6410535865506
122700000 52.20262776503435
122800000 49.76672582407041
122900000 50.44649994757267
123000000 60.904608780360356
123100000 55.32

...

639000000 1.7395262670871252
639100000 2.442925355131397
639200000 1.4052261709741911
639300000 1.810359281946906
best so far: 0
type: [1, 1, 2, 13, 5] 130
cases of this type: 5492500
639400000 2.033052978186644
639500000 1.5086166211492795
639600000 1.185559839741154
639700000 1.4643061138719498
639800000 1.8963107061595736
639900000 1.6908056974412131
640000000 1.5248917328547436
640100000 1.4185522373287103
640200000 1.2225225780861868
640300000 1.8355561769497057
640400000 1.820121001136001
640500000 1.872581633580881
640600000 1.5444082306726126
640700000 1.3258939960312843
640800000 0.9020109329332484
640900000 1.315042951918655
641000000 0.9714327473631998
641100000 0.9274812536167907
641200000 0.8793132000077115
641300000 1.0460934427924453
641400000 1.7704104270968417
641500000 1.4622470406988273
641600000 1.4027081825154006
641700000 1.2105035835325229
641800000 1.1473686380360113
641900000 1.3352870025700212
642000000 1.6001079575091466
642100000 0.8865221523832593
642200000

665800000 0.44870839565989173
665900000 0.398784484472388
666000000 0.3753185744908849
666100000 0.4779435583413389
666200000 0.38768203419669667
666300000 0.39116885906164184
666400000 1.6812962566502954
666500000 3.6229991557015
666600000 3.5346415601686605
666700000 2.276230154006528
666800000 2.3648630085969153
666900000 2.2670370756033282
667000000 2.3981211212438454
667100000 2.472511578544712
667200000 2.1413146580258022
best so far: 0
type: [1, 1, 10, 13, 1] 130
cases of this type: 219700
667300000 1.0018283400188235
667400000 0.3377266299342652
best so far: 0
type: [1, 1, 13, 1, 10] 130
cases of this type: 1690000
667500000 1.3254096694844515
667600000 2.646317731586095
667700000 2.0489349282745954
667800000 1.6943935532507794
667900000 2.1241161752200215
668000000 1.096272457723679
668100000 0.34301782159147987
668200000 0.3429053695039802
668300000 0.3431146605288366
668400000 0.34166638570170005
668500000 0.34673088954521414
668600000 0.3500545394270599
668700000 0.34663587

691600000 1.5822836655384873
691700000 0.8688682835218152
691800000 0.5857490258320649
691900000 1.2507018394056166
692000000 1.394621836986047
692100000 1.4644144049695114
692200000 1.1988487918706476
692300000 0.6020774360132614
692400000 0.9939530371090505
692500000 1.3011914232540172
692600000 1.3506543247787872
692700000 1.613417602115342
692800000 1.0179603797683867
692900000 0.7483640613877297
693000000 1.2089174975453303
693100000 1.6874151247819096
693200000 1.6360505797251084
693300000 1.0601832248527698
693400000 0.7630697034123519
693500000 1.3776206507180813
693600000 1.280298208694754
693700000 1.2523151259742678
693800000 1.1271147410127105
693900000 0.8153528696690798
694000000 0.6981520537524177
694100000 1.0994275237591307
694200000 1.5626027790797619
694300000 1.994543455619124
694400000 1.2156531120394845
694500000 0.8157044969078376
694600000 1.0109679898529378
694700000 1.8892504533222914
694800000 1.2783358097057607
694900000 1.3857292193024218
695000000 0.792433

718700000 0.3346499096203824
best so far: 0
type: [1, 2, 13, 1, 5] 130
cases of this type: 211250
718800000 0.11024778962376248
718900000 0.12744618969122504
719000000 0.18638935108626484
best so far: 0
type: [1, 2, 13, 5, 1] 130
cases of this type: 42250
best so far: 0
type: [1, 2, 65, 1, 1] 130
cases of this type: 8450
best so far: 0
type: [1, 5, 1, 1, 26] 130
cases of this type: 2284880
719100000 0.26112288608299294
719200000 0.3886605091805346
719300000 0.347887178708301
719400000 0.3371773155221475
719500000 0.4630458629631294
719600000 0.3367545056076228
719700000 0.32625414118523666
719800000 0.30678310411954923
719900000 0.2778057073971808
720000000 0.33466227369077933
720100000 0.1800660288801339
720200000 0.1194808860669891
720300000 0.12195511370064484
720400000 0.12541970953493117
720500000 0.28337746858807006
720600000 0.3267972735132582
720700000 0.33019511760149134
720800000 0.3935641617952108
720900000 0.3538457033122208
721000000 0.38347909468524594
721100000 0.3000494

739200000 0.1914197911016199
739300000 0.1537063526291258
739400000 0.1809301717626254
739500000 0.1764297413011829
739600000 0.1629017670761731
739700000 0.15703770766240954
739800000 0.15204610956272085
739900000 0.15956613434404332
740000000 0.17223939002763033
740100000 0.15710234445359575
740200000 0.17303418284370567
740300000 0.17716460981748103
740400000 0.16191337041552398
740500000 0.1558330384726617
740600000 0.2228603083854795
740700000 0.18193092221022514
740800000 0.13391965317616528
740900000 0.14764632405477762
741000000 0.18526799616504114
741100000 0.1311052027641846
741200000 0.13633707467056513
741300000 0.13891020407079988
741400000 0.16473193197420768
741500000 0.14437869404796957
741600000 0.14385138193695016
741700000 0.16972112859907218
741800000 0.1437511199115654
best so far: 0
type: [2, 1, 1, 5, 13] 130
cases of this type: 3570125
741900000 0.1012127227315704
742000000 0.04677197278310392
742100000 0.05546748076641758
742200000 0.06597475805985066
742300000 